## car_lane_detection_shadow
### 2019-1 과제연구
#### : 그림자가 있을 때 차선을 인식하도록 개선한 코드 (사진)

***

In [42]:
import numpy as np
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from PIL import Image

%matplotlib auto

Using matplotlib backend: Qt5Agg


In [3]:
def make_coordinates(image, line_parameters):
    slope, intercept = line_parameters
    #기울기   절편
    try:
        y1 = image.shape[0] #height
        y2 = int(y1*(3/5)) #height의 3/5까지 올라갈 때까지 탐색...
        x1 = int((y1 - intercept)/slope)
        x2 = int((y2 - intercept)/slope)
        return np.array([x1, y1, x2, y2])
    except OverflowError:
        return np.array([0, 0, 0, 0])

In [4]:
def average_slope_intercept(image, lines):
    left_fit = [] #왼쪽 line 의 평균
    right_fit = [] #오른쪽 line의 평균좌표
    try:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            parameters = np.polyfit((x1, x2), (y1, y2), 1)
            slope = parameters[0]
            intercept = parameters[1]

            if slope < -0.6: #line의 기울기가 음수인 경우
                left_fit.append((slope, intercept))
                #print('left slope', slope)
            if 18 > slope > 0.6: #line의 기울기가 양수인 경우
                right_fit.append((slope, intercept))
            if slope >= 18:
                print("can't detect!")
            cnt = 0 #cnt 초기화

    except TypeError:
        left_fit_average = []
        right_fit_average = []
    
    #위 모든 값들을 평균하여서 기울기와 y절편 계산하기
    left_fit_average = np.average(left_fit, axis=0)
    right_fit_average = np.average(right_fit, axis=0)
    
    if left_fit_average != []:
        left_line = make_coordinates(image, left_fit_average)
    else: 
        left_line = np.array([0, 0, 0, 0])
    if right_fit_average != []:
        right_line = make_coordinates(image, right_fit_average)
    else:
        right_line = np.array([0, 0, 0, 0])
    #print(left_fit)
    return np.array([left_line, right_line])


In [5]:
def canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0) 
    canny = cv2.Canny(blur, 10, 150)
    return canny

In [6]:
def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None: # ==not empty
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4) #2d array 를 1d array 로 변환
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    return line_image

In [13]:
def region_of_interest(image):
    height = image.shape[0] 
    width = image.shape[1]
    mwidth = width / 2
    polygons = np.array([
        [(int(width * 1 / 9), height), (int(width * 8 / 9), height), (int(mwidth), int(height * 3/9))]
    ])
    
    mask = np.zeros_like(image) #이미지와 같은 모양(pixel)으로 0배열을 만든다.
    cv2.fillPoly(mask, polygons, 255) #윤곽 만들기
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

### 그림자 있을때 인식

In [109]:
def amplify(image):
    s = image.shape
    #img = image.shape
    width = s[1]
    height = s[0]
    im = cv2.imread('test_image_.jpg')

    for j in range(int(width)):
        for i in range(int(height)):
            bgr = im[int(i), int(j)]
            if bgr[2] > 55 and bgr[2] < 120: im[int(i)][int(j)] = [255, 255, 255]
            #except IndexError: 
    return im
    
#bgr순서
frame = cv2.imread('test_image_.jpg')
img = amplify(frame)
lane_image = np.copy(frame)
canny_img = canny(img)
cropped_image = region_of_interest(canny_img)
cv2.imwrite('pic2.jpg', img)

#Hough Transform (Region of Interest) and draw lines
lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]),minLineLength = 100, maxLineGap = 5)
averaged_lines = average_slope_intercept(lane_image, lines)
line_image = display_lines(lane_image, averaged_lines)
#averaged_lines를 통해서 line을 조금 더 부드럽게 만들기
combo_image = cv2.addWeighted(lane_image, 0.8, line_image, 1, 1) 
              #가중치 부여해 이미지 합치기
    
cv2.imshow("result", combo_image)
cv2.waitKey(0)

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


-1

### 그냥 인식

In [108]:
frame = cv2.imread('test_image_.jpg')
lane_image = np.copy(frame)
canny_img = canny(frame)
cropped_image = region_of_interest(canny_img)

#Hough Transform (Region of Interest) and draw lines
lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]),minLineLength = 100, maxLineGap = 5)
averaged_lines = average_slope_intercept(lane_image, lines)
line_image = display_lines(lane_image, averaged_lines)
#averaged_lines를 통해서 line을 조금 더 부드럽게 만들기
combo_image = cv2.addWeighted(lane_image, 0.8, line_image, 1, 1) 
              #가중치 부여해 이미지 합치기
    
cv2.imshow("result", combo_image)
cv2.waitKey(0)

C:\Users\User\Anaconda3\lib\site-packages\numpy\lib\function_base.py:392: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\User\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


113

***
### Calculating RGB Averarge (K-Means Clustering

In [113]:
image_path = "test_rgb_2.jpg"
avg = [0, 0, 0]
image = cv2.imread(image_path)
s = image.shape
width = s[1]
height = s[0]

for j in range(int(width)):
    for i in range(int(height)):
        bgr = image[int(i), int(j)]
        avg[0] += bgr[2]
        avg[1] += bgr[1]
        avg[2] += bgr[0]
for i in range(3):
    avg[i] = avg[i] / (width * height)
print(avg)

[132.61586558421746, 125.41924898833015, 128.75397289875298]
